<a href="https://colab.research.google.com/github/Tahir7044/machine-learning/blob/master/santanzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# feature selection by filtering method

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import VarianceThreshold 

In [19]:
!pip uninstall kaggle

Uninstalling kaggle-1.5.6:
  Would remove:
    /usr/local/bin/kaggle
    /usr/local/lib/python3.6/dist-packages/kaggle-1.5.6.dist-info/*
    /usr/local/lib/python3.6/dist-packages/kaggle/*
Proceed (y/n)? y
  Successfully uninstalled kaggle-1.5.6


In [2]:
!kaggle -v

Kaggle API 1.5.6


In [5]:
!kaggle competitions download -c santander-customer-transaction-prediction

100% 249M/250M [00:07<00:00, 29.5MB/s]
100% 250M/250M [00:07<00:00, 34.5MB/s]


In [13]:
!kaggle datasets download -d ioramishvili/santaner

 73% 9.00M/12.3M [00:01<00:00, 7.78MB/s]
100% 12.3M/12.3M [00:01<00:00, 10.3MB/s]


In [0]:
!unzip /content/santander-customer-transaction-prediction.zip

In [0]:
!unzip /content/santaner.zip

In [0]:
data_set=pd.read_csv("/content/santanzer_dataset/santanzer_train.csv")

In [20]:
data_set.head()

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,imp_op_var41_comer_ult1,imp_op_var41_comer_ult3,imp_op_var41_efect_ult1,imp_op_var41_efect_ult3,imp_op_var41_ult1,imp_op_var39_efect_ult1,imp_op_var39_efect_ult3,imp_op_var39_ult1,imp_sal_var16_ult1,ind_var1_0,ind_var1,ind_var2_0,ind_var2,ind_var5_0,ind_var5,ind_var6_0,ind_var6,ind_var8_0,ind_var8,ind_var12_0,ind_var12,ind_var13_0,ind_var13_corto_0,ind_var13_corto,ind_var13_largo_0,ind_var13_largo,ind_var13_medio_0,ind_var13_medio,ind_var13,...,saldo_medio_var5_ult1,saldo_medio_var5_ult3,saldo_medio_var8_hace2,saldo_medio_var8_hace3,saldo_medio_var8_ult1,saldo_medio_var8_ult3,saldo_medio_var12_hace2,saldo_medio_var12_hace3,saldo_medio_var12_ult1,saldo_medio_var12_ult3,saldo_medio_var13_corto_hace2,saldo_medio_var13_corto_hace3,saldo_medio_var13_corto_ult1,saldo_medio_var13_corto_ult3,saldo_medio_var13_largo_hace2,saldo_medio_var13_largo_hace3,saldo_medio_var13_largo_ult1,saldo_medio_var13_largo_ult3,saldo_medio_var13_medio_hace2,saldo_medio_var13_medio_hace3,saldo_medio_var13_medio_ult1,saldo_medio_var13_medio_ult3,saldo_medio_var17_hace2,saldo_medio_var17_hace3,saldo_medio_var17_ult1,saldo_medio_var17_ult3,saldo_medio_var29_hace2,saldo_medio_var29_hace3,saldo_medio_var29_ult1,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
0,1,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.170000,0
1,3,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,1,...,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,300.0,122.22,300.0,240.75,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.030000,0
2,4,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,3.00,2.07,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.770000,0
3,8,2,37,0.0,195.0,195.0,0.0,0.0,0.0,0.0,0.0,195.0,195.0,0.0,0.0,195.0,0.0,0.0,195.0,0.0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,91.56,138.84,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64007.970000,0
4,10,2,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,...,40501.08,13501.47,0.0,0.0,0.0,0.0,0.0,0.0,85501.89,85501.89,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117310.979016,0


In [21]:
print(data_set.shape)

(76020, 371)


In [24]:
data_set['TARGET'].value_counts()

0    73012
1     3008
Name: TARGET, dtype: int64

In [26]:
x=data_set.drop('TARGET',axis=1)
y=data_set['TARGET']
print(x.shape,y.shape)

(76020, 370) (76020,)


In [0]:
 x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=0,stratify=y)

In [29]:
#constant feature removal
constant_filter=VarianceThreshold(threshold=0)
constant_filter.fit(x_train)

VarianceThreshold(threshold=0)

In [31]:
constant_filter.get_support().sum()

334

In [0]:
constant_list=[not temp for temp in constant_filter.get_support()]

In [0]:
x_train.columns[constant_list]

In [40]:
x_train_filter = constant_filter.transform(x_train)
x_test_filter = constant_filter.transform(x_test)
print(x_train_filter.shape,x_test_filter.shape)

(57015, 334) (19005, 334)


In [43]:
# quasi constant feature removal
quasi_constant_filter=VarianceThreshold(threshold=0.01)
quasi_constant_filter.fit(x_train_filter)

VarianceThreshold(threshold=0.01)

In [44]:
quasi_constant_filter.get_support().sum()

273

In [0]:
quasi_constant_list=[not temp for temp in quasi_constant_filter.get_support()]


In [48]:
x_train_quasi_filter = quasi_constant_filter.transform(x_train_filter)
x_test_quasi_filter = quasi_constant_filter.transform(x_test_filter)
print(x_train_quasi_filter.shape)
print(x_test_quasi_filter.shape)

(57015, 273)
(19005, 273)


In [0]:
#duplicate features removal
x_train_T = x_train_quasi_filter.T
x_test_T = x_test_quasi_filter.T

In [0]:
x_train_T=pd.DataFrame(x_train_T)
x_test_T=pd.DataFrame(x_test_T)


In [52]:
print(x_test_T.shape,x_train_T.shape)
print(type(x_train_T))

(273, 19005) (273, 57015)
<class 'pandas.core.frame.DataFrame'>


In [53]:
x_train_T.duplicated().sum()

17

In [0]:
duplicated_features = x_train_T.duplicated()
features_to_keep = [ not index for index in duplicated_features]
x_train_unique = x_train_T[features_to_keep].T
x_test_unique = x_test_T[features_to_keep].T

In [59]:
print(x_train_unique.shape,x_test_unique.shape)
print(y_train.shape,y_test.shape)

(57015, 256) (19005, 256)
(57015,) (19005,)


###performance comparasion

In [0]:
def run_randomforest(x_train,x_test,y_train,y_test):
  clf = RandomForestClassifier(n_estimators=100,random_state=0,n_jobs=-1)
  clf.fit(x_train,y_train)
  y_pred = clf.predict(x_test)
  print("accuracy on test set:")
  print(accuracy_score(y_test,y_pred))

In [64]:
%%time
run_randomforest(x_train_unique,x_test_unique,y_train,y_test)

accuracy on test set:
0.9577479610628782
CPU times: user 21.9 s, sys: 47.1 ms, total: 22 s
Wall time: 11.3 s


In [65]:
%%time
run_randomforest(x_train,x_test,y_train,y_test)

accuracy on test set:
0.9578531965272297
CPU times: user 25.3 s, sys: 74 ms, total: 25.3 s
Wall time: 13 s


###correlation method for feature selection